<a href="https://colab.research.google.com/github/jiruneko/New_RAG/blob/main/New_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
import os
import requests
import io
from urllib.parse import urlparse
from PyPDF2 import PdfReader, PdfWriter

def get_pdf_reader(input_sourse):
  base_filename = "output"
  response = requests.get(input_sourse, stream=True, timeout=30)
  response.raise_for_status()

  # URLパスからファイル名を取得
  parsed_url = urlparse(input_sourse)
  path_part = os.path.basename(parsed_url.path)
  if path_part and '.' in path_part:
    base_filename = os.path.splitext(path_part)[0]

  # コンテンツをメモリに読み込む
  pdf_content = io.BytesIO(response.content)
  reader = PdfReader(pdf_content)
  total_pages = len(reader.pages)
  return reader, base_filename, total_pages

In [ ]:
def split_pdf(input_sourse, output_dir, pages_per_chunk):
  reader, base_filename, total_pages = get_pdf_reader(input_sourse)

  if reader is None:
    print("PDFリーダーの取得に失敗しました。分割を中止しています。")
    return

  try:
    # 出力ディレクトリが存在しない場合は作成します
    os.makedirs(output_dir, exist_ok=True)
    print(f"出力ディレクトリ '{output_dir}' が確保されました。")

    # チャンクの数を計算します
    num_chunks = (total_pages + pages_per_chunk - 1)
    print(f"Splitting into {num_chunks} chunks of max {pages_per_chunk} pages each.")

    # 各チャンクを処理します
    for i in range(num_chunks):
      writer = PdfWriter()
      start_page = i * pages_per_chunk
      end_page = min(start_page + pages_per_chunk, total_pages)

      print(f"Processing chunk {i + 1}/{num_chunks} (pages {start_page + 1} - {end_page})...")

      for page_num in range(start_page, end_page):
        writer.add_page(reader.pages[page_num])

      output_filename = os.path.join(output_dir, f"{base_filename}_chunk_{i+1}.pdf")

      with open(output_filename, 'wb') as outfile:
        writer.write(outfile)
      print(f"Chunk {i+1} saved as '{output_filename}'")

    print("\nPDF splitting completed successfully!")
  except Exception as e:
    print(f"An error occurred during the splitting process: {e}")

In [ ]:
query = "NLPでTransformerモデルを使用する主な利点は何ですか？"
documents = [
    "リカレントニューラルネットワーク（RNN）は、以前はシーケンスタスクでよく使用されていました。",
    "Transformerは入力トークンの並列処理を可能にするため、RNNと比較してトレーニング時間が短縮されます。",
    "人気のTransformerモデルであるBERTは、多くのNLPベンチマークで最先端の結果を達成しています。",
    "Transformerのアテンションメカニズムにより、モデルは入力シーケンス内の異なる単語の重要度を重み付けできます。",
    "畳み込みニューラルネットワーク（CNN）は主にコンピュータービジョンで使用されます。",
    "Transformerアーキテクチャの主な利点は、RNNよりも長距離の依存関係をより効果的に処理できることです。",
    "特定の下流タスクに合わせて事前トレーニング済みのTransformerモデルを微調整できます。"
]

In [ ]:
query = "NLPでTransformerモデルを使用する主な利点は何ですか？"
documents = [
    "リカレントニューラルネットワーク（RNN）は、以前はシーケンスタスクでよく使用されていました。",
    "Transformerは入力トークンの並列処理を可能にするため、RNNと比較してトレーニング時間が短縮されます。",
    "人気のTransformerモデルであるBERTは、多くのNLPベンチマークで最先端の結果を達成しています。",
    "Transformerのアテンションメカニズムにより、モデルは入力シーケンス内の異なる単語の重要度を重み付けできます。",
    "畳み込みニューラルネットワーク（CNN）は主にコンピュータービジョンで使用されます。",
    "Transformerアーキテクチャの主な利点は、RNNよりも長距離の依存関係をより効果的に処理できることです。",
    "特定の下流タスクに合わせて事前トレーニング済みのTransformerモデルを微調整できます。"
]

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder
model = CrossEncoder("BAAI/bge-reranker-v2-m3")
sentence_pairs = [[query, doc] for doc in documents]
scores = model.predict(sentence_pairs, show_progress_bar=True)

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/393 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
docs_with_scores = list(zip(documents, scores))
reranked = sorted(docs_with_scores, key=lambda x: x[1], reverse=True)
print("\n--- Reranked Document Order ---")
print("(Higher score indicates higher relevance)")
for i, (doc, score) in enumerate(reranked):
  print(f"{i+1}. Score: {score:4f} - {doc}")


--- Reranked Document Order ---
(Higher score indicates higher relevance)
1. Score: 0.950797 - Transformerアーキテクチャの主な利点は、RNNよりも長距離の依存関係をより効果的に処理できることです。
2. Score: 0.257916 - 人気のTransformerモデルであるBERTは、多くのNLPベンチマークで最先端の結果を達成しています。
3. Score: 0.117288 - Transformerのアテンションメカニズムにより、モデルは入力シーケンス内の異なる単語の重要度を重み付けできます。
4. Score: 0.095247 - Transformerは入力トークンの並列処理を可能にするため、RNNと比較してトレーニング時間が短縮されます。
5. Score: 0.080434 - 特定の下流タスクに合わせて事前トレーニング済みのTransformerモデルを微調整できます。
6. Score: 0.000788 - 畳み込みニューラルネットワーク（CNN）は主にコンピュータービジョンで使用されます。
7. Score: 0.000088 - リカレントニューラルネットワーク（RNN）は、以前はシーケンスタスクでよく使用されていました。


In [ ]:
!pip install llama-index llama-index-llms-openai

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import os
from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.llms.openai import OpenAI

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# 2. サンプルデータの作成（安全な内容に置き換えています）
# RAGの学習用として、料理のレシピやプログラミングの手順などを想定してください
docs = [
    Document(text="美味しいカレーを作るには、まず玉ねぎを飴色になるまで炒めます。"),
    Document(text="次にスパイスと具材を加え、じっくり煮込むことでコクが出ます。"),
]

# 3. モデルの設定（gpt-3.5-turboを使用）
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

# 4. インデックスの作成（ドキュメントをベクトル化して保存）
index = VectorStoreIndex.from_documents(docs)

# 5. クエリエンジンの作成
query_engine = index.as_query_engine(response_mode="compact")

# 6. 質問の実行
response = query_engine.query("カレーを美味しく作るコツは何ですか？")
print(response)

NameError: name 'OPENAI_API_KEY' is not defined